In [12]:
import numpy as np
from torch import nn,optim
from torch.autograd import Variable
import torch
from torchvision import datasets, transforms # 也是torch的工具，里面封装了更高级的功能
from torch.utils.data import DataLoader

In [13]:
# 训练集
train_dataset = datasets.MNIST(root='./', # 这里选择当前路径下载，意思就是说可以在当前路径MINST文件夹内找到数据，数据是封装好了的
                            train = True, # 下载训练集
                            transform = transforms.ToTensor(), #把数据变成基本的数据类型tensor
                            download = True) # 直接从封装好接口上载入

# 测试集
test_dataset = datasets.MNIST(root='./', # 这里选择当前路径下载
                            train = False, # 下载训练集
                            transform = transforms.ToTensor(), #把数据变成基本的数据类型tensor
                            download = True) # 直接从封装好接口上载入

train_dataset # 这个东西已经被封装成了60000个点，使用Data_loader拿出来用就好
len(train_dataset)


60000

In [14]:
# 批次大小，每次训练选择多少的数据
batch_size = 64 

# 装载训练集
# dataloader相当于创建了一个数据生成器
# 在这个里面数据就已经封装好了（train_loader和test_loader里面）
train_loader = DataLoader(dataset = train_dataset,  # 数据的来源
                          batch_size = batch_size,  # 批次的大小
                          shuffle = True) # 数据是否进行打乱

test_loader = DataLoader(dataset = test_dataset, # 来源于测试集
                         batch_size = batch_size,
                         shuffle = True)

train_loader

In [15]:
# 数据生成器使用方法——在循环中进行使用
for i,data in enumerate(train_loader): #enumerate是为了每一批次的数据增加一个索引i
    inputs, labels = data # data里面包含了要输入的量和输出的标签值
    print(inputs.shape)
    print(labels.shape)
    break
print(labels)
# 64个样本 1个通道（黑白） 28*28的像素
# 标签同样是64个，标签值代表了是哪个数据
len(train_loader) # 938*64 = 60032 刚好枚举完

torch.Size([64, 1, 28, 28])
torch.Size([64])
tensor([0, 8, 4, 6, 9, 4, 4, 6, 9, 9, 9, 1, 8, 3, 3, 0, 6, 6, 9, 7, 3, 9, 5, 3,
        5, 0, 7, 2, 9, 2, 7, 8, 6, 1, 6, 3, 0, 3, 9, 7, 1, 9, 4, 4, 3, 9, 8, 7,
        3, 4, 1, 2, 9, 7, 2, 1, 2, 1, 1, 1, 1, 4, 7, 8])


938

# 对以下网络的一些解释
定义在网络结构里面的每一个背后都内含参数的
全连接层会对不同的样本进行分别处理
但是softmax是可以跨样本进行处理的（可能为了后续一些算法？）
如果dim=0，则会对每一列进行归一化，不符合我们要求，所以要dim=1，完成行归一化

# Dropout的相应用法
Dropout一般在隐藏层中使用，编写代码的时候写在网络结构层编写
在写网络结构时，可以使用封装代码
非封装：self.fc1 = nn.Linear(784,500)
封装版：self.layer1 = nn.Sequential(nn.Linear(784,500), nn.Dropout(p=0.5), nn.Tanh())
然后调用的时候直接用layer1即可

In [16]:
# 定义网络结构 Pytorch，一个网络结构（初始化） 一个网络计算（得到结果）
# 如果要用 Dropout，就要使用多层的网络结构

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() # 是方法函数就需要用()
        self.layer1 = nn.Sequential(nn.Linear(784,500), nn.Dropout(p=0.5), nn.Tanh())
        self.layer2 = nn.Sequential(nn.Linear(500,300), nn.Dropout(p=0.5), nn.Tanh()) # 隐藏层是超参数
        self.layer3 = nn.Sequential(nn.Linear(300,10), nn.Softmax(dim=1)) # 输出层不用Dropout，这些nn的方法第一个字母大写

    def forward(self, x):
        # x 输入的时候维度是[64,1,28,28], 全连接层只能处理2维数据，第一个维度是样本索引，第二个维度是样本内的
        # ([64, 1, 28, 28])->(64, 784)
        x = x.reshape(x.size()[0],-1) #-1代表自适应，变成64行，784列
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [17]:
# 定义模型（对象化）
model = Net() #对象化也是一个方法
# 定义代价函数
# 这是一个类，你需要实例化这个类！
mse_loss = nn.CrossEntropyLoss() #交叉熵损失
# 定义优化器
# 要传入模型参数和学习率
LR = 0.5
optimizer = optim.SGD(model.parameters(), LR)

In [18]:
# 进行模型的训练和测试方法——一边训练一边测试模型效果
def train():
    # 在dropout里面需要额外加上model.train()
    model.train() # 代表是训练状态，此时模型的dropout是起作用的
    for i,data in enumerate(train_loader): # 对迭代器进行分析
        # 循环一次，就会获得一次数据的批次和标签，这很重要！
        inputs, labels = data
        # 获得模型预测结果, 这个结果是[64, 10]的一个矩阵，64是批次，10是概率值
        out = model(inputs)
        
        # 用交叉熵损失，用交叉损失时，函数会内部自动转换成独热编码
        # 其中，out是（64，10）数组，labels是64的一位数组
        # 根据这个函数的用法，两者不需要对齐，只要保证第一个数都是minibatch值即可
        loss = mse_loss(out, labels)
        
        # 梯度清零
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 修改权值
        optimizer.step()

def test():
    model.eval()
    # 测试集准确率
    correct = 0
    for i,data in enumerate(test_loader): # 每一次都是在对一个训练好的模型进行评估，把所有的测试集跑了一边
        inputs, labels = data
        out = model(inputs)
        # torch.max计算完后得到两个值，获得最大值和最大值所在的位置，dim=1代表对每一行进行操作
        # 这里会对64行每一行都进行操作的
        _, predicted = torch.max(out, 1)
        # 正确个数
        correct += (predicted == labels).sum() # 这里面的true和false会一个一个对比
    print('Test acc:{0}'.format(correct.item()/len(test_dataset)))

    # 训练集准确率
    correct = 0
    for i,data in enumerate(train_loader): # 每一次都是在对一个训练好的模型进行评估，把所有的测试集跑了一边
        inputs, labels = data
        out = model(inputs)
        # torch.max计算完后得到两个值，获得最大值和最大值所在的位置，dim=1代表对每一行进行操作
        # 这里会对64行每一行都进行操作的
        _, predicted = torch.max(out, 1)
        # 正确个数
        correct += (predicted == labels).sum() # 这里面的true和false会一个一个对比
    print('Train acc:{0}'.format(correct.item()/len(train_dataset)))

# Dropout 使用场景
并不一定让准确度变高，在模型比较复杂的情况下，应当使用，防止过拟合

In [19]:
# 循环运行
# 获得了更快的收敛速度
for epoch in range(20):
    print('epoch:', epoch)
    train()
    test()

epoch: 0
Test acc:0.9174
Train acc:0.9161833333333333
epoch: 1
Test acc:0.923
Train acc:0.9253666666666667
epoch: 2
Test acc:0.9392
Train acc:0.94135
epoch: 3
Test acc:0.9449
Train acc:0.9462333333333334
epoch: 4
Test acc:0.9463
Train acc:0.9490833333333333
epoch: 5
Test acc:0.9472
Train acc:0.95165
epoch: 6
Test acc:0.9536
Train acc:0.9582666666666667
epoch: 7
Test acc:0.9566
Train acc:0.962
epoch: 8
Test acc:0.9563
Train acc:0.9627333333333333
epoch: 9
Test acc:0.9572
Train acc:0.9638833333333333
epoch: 10
Test acc:0.9618
Train acc:0.9663166666666667
epoch: 11
Test acc:0.962
Train acc:0.9673333333333334
epoch: 12
Test acc:0.9637
Train acc:0.9698166666666667
epoch: 13
Test acc:0.9609
Train acc:0.96905
epoch: 14
Test acc:0.9642
Train acc:0.9717333333333333
epoch: 15
Test acc:0.9638
Train acc:0.9713666666666667
epoch: 16
Test acc:0.9641
Train acc:0.9714333333333334
epoch: 17
Test acc:0.9656
Train acc:0.9737
epoch: 18
Test acc:0.9644
Train acc:0.9733166666666667
epoch: 19
Test acc:0.9675

# 这里是一个常见的框架
数据集的载入
定义批次大小和分割数据集（接下去在循环中对这个数据进行训练）
定义网络结构 和 网络运算 def __init__(self) 和 def forward(self, x)
对象化模型 代价函数 优化器
写网络如何训练测试——用两个模块封装好
训练：输出结果 求损失 梯度归零 更新参数
测试：输出结果

一个大循环